In [24]:
import pandas as pd
import numpy as np
import math

In [185]:
def prettyTable(df, col):
    a = df[col].value_counts(dropna = False)
    b = df[col].value_counts(normalize = True, dropna = False)
    c = pd.concat([a, b], axis = 1)
    c.columns = [col + '_count', col + '_ratio']
    return c

In [1]:
# cash flow simulations

In [ ]:
https://www.occ.treas.gov/publications-and-resources/publications/comptrollers-handbook/files/asset-securitization/pub-ch-asset-securitization.pdf
    page 54

### Sim 1 - Payments on One Loan

In [28]:
# sim 1 

loan_start_date = '09-01-2020'
loan_maturity_date = '08-31-2050'
# (pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) // np.timedelta64(1, 'M')
# math.ceil((pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) / np.timedelta64(1, 'M'))
number_of_pds = 360

loan_amnt = 500000
annual_ir = .045
monthly_ir = annual_ir / 12

In [119]:
# (pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) // np.timedelta64(1, 'M')
# math.ceil((pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) / np.timedelta64(1, 'M'))

360

In [39]:
# create dates

dates_index = pd.date_range(loan_start_date, loan_maturity_date, freq = 'M')

In [41]:
df = pd.DataFrame(columns = ['Period', 'Payment', 'Interest', 'Principal', 'Residual Balance'])
df

,Period,Payment,Interest,Principal,Residual Balance


In [42]:
df['Period'] = dates_index
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,NaN,NaN,NaN,NaN
1,2020-10-31,NaN,NaN,NaN,NaN
2,2020-11-30,NaN,NaN,NaN,NaN
3,2020-12-31,NaN,NaN,NaN,NaN
4,2021-01-31,NaN,NaN,NaN,NaN
...,...,...,...,...,...
355,2050-04-30,NaN,NaN,NaN,NaN
356,2050-05-31,NaN,NaN,NaN,NaN
357,2050-06-30,NaN,NaN,NaN,NaN
358,2050-07-31,NaN,NaN,NaN,NaN


In [47]:
# monthly payment formula
# M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]

monthly_payment = loan_amnt * ((monthly_ir * (monthly_ir + 1) ** number_of_pds) / (((monthly_ir + 1) ** (number_of_pds)) - 1))
monthly_payment = round(monthly_payment, 2)
monthly_payment

2533.43

In [48]:
df['Payment'] = monthly_payment

In [49]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,NaN,NaN,NaN
1,2020-10-31,2533.43,NaN,NaN,NaN
2,2020-11-30,2533.43,NaN,NaN,NaN
3,2020-12-31,2533.43,NaN,NaN,NaN
4,2021-01-31,2533.43,NaN,NaN,NaN
...,...,...,...,...,...
355,2050-04-30,2533.43,NaN,NaN,NaN
356,2050-05-31,2533.43,NaN,NaN,NaN
357,2050-06-30,2533.43,NaN,NaN,NaN
358,2050-07-31,2533.43,NaN,NaN,NaN


In [50]:
loan_amnt * monthly_ir

1875.0

In [52]:
round(monthly_payment - (loan_amnt * monthly_ir), 2)

658.43

In [53]:
loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)

499341.57

In [58]:
initial_payment = [loan_amnt * monthly_ir, round(monthly_payment - (loan_amnt * monthly_ir), 2), loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)]

In [59]:
initial_payment

[1875.0, 658.43, 499341.57]

In [60]:
df.loc[0, ['Interest', 'Principal', 'Residual Balance']]

Interest            NaN
Principal           NaN
Residual Balance    NaN
Name: 0, dtype: object

In [61]:
df.loc[0, ['Interest', 'Principal', 'Residual Balance']] = initial_payment

In [62]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,NaN,NaN,NaN
2,2020-11-30,2533.43,NaN,NaN,NaN
3,2020-12-31,2533.43,NaN,NaN,NaN
4,2021-01-31,2533.43,NaN,NaN,NaN
...,...,...,...,...,...
355,2050-04-30,2533.43,NaN,NaN,NaN
356,2050-05-31,2533.43,NaN,NaN,NaN
357,2050-06-30,2533.43,NaN,NaN,NaN
358,2050-07-31,2533.43,NaN,NaN,NaN


In [64]:
df.loc[1, ]

Period              2020-10-31 00:00:00
Payment                         2533.43
Interest                            NaN
Principal                           NaN
Residual Balance                    NaN
Name: 1, dtype: object

In [99]:
int_vals = []
prin_vals = []
bal_vals = []

for i in df.index[1:, ]:
    interest = round(monthly_ir * df.iloc[i-1]['Residual Balance'], 2)
    principal = round(df.iloc[i]['Payment'] - interest, 2)
    res_bal = round(df.iloc[i-1]['Residual Balance'] - principal, 2)
    df.loc[i, ['Interest', 'Principal', 'Residual Balance']] = [interest, principal, res_bal]
    int_vals.append(interest)
    prin_vals.append(principal)
    bal_vals.append(res_bal)

In [101]:
# int_vals
# prin_vals
# bal_vals

In [102]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,1872.53,660.9,498680.67
2,2020-11-30,2533.43,1870.05,663.38,498017.29
3,2020-12-31,2533.43,1867.56,665.87,497351.42
4,2021-01-31,2533.43,1865.07,668.36,496683.06
...,...,...,...,...,...
355,2050-04-30,2533.43,46.96,2486.47,10036.95
356,2050-05-31,2533.43,37.64,2495.79,7541.16
357,2050-06-30,2533.43,28.28,2505.15,5036.01
358,2050-07-31,2533.43,18.89,2514.54,2521.47


In [113]:
# df['Residual Balance'] = df['Residual Balance'].clip(lower = 0)

In [114]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,1872.53,660.9,498680.67
2,2020-11-30,2533.43,1870.05,663.38,498017.29
3,2020-12-31,2533.43,1867.56,665.87,497351.42
4,2021-01-31,2533.43,1865.07,668.36,496683.06
...,...,...,...,...,...
355,2050-04-30,2533.43,46.96,2486.47,10036.95
356,2050-05-31,2533.43,37.64,2495.79,7541.16
357,2050-06-30,2533.43,28.28,2505.15,5036.01
358,2050-07-31,2533.43,18.89,2514.54,2521.47


In [115]:
display(df['Payment'].sum())
display(df['Interest'].sum())
display(df['Principal'].sum())

912034.8

412032.29999999993

500002.50000000006

In [116]:
df['Residual Balance'].clip(lower = 0)

0      499341.57
1      498680.67
2      498017.29
3      497351.42
4      496683.06
         ...    
355     10036.95
356      7541.16
357      5036.01
358      2521.47
359            0
Name: Residual Balance, Length: 360, dtype: object

### Automating

In [126]:
# making functions to run the sim

#### Assumptions and Inputs

In [127]:
# Assumptions


loan_start_date = '09-01-2020'
loan_maturity_date = '08-31-2050'

loan_amnt = 500000
annual_ir = .045


number_of_pds = math.ceil((pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) / np.timedelta64(1, 'M'))
monthly_ir = annual_ir / 12

In [128]:
# create dates

dates_index = pd.date_range(loan_start_date, loan_maturity_date, freq = 'M')

In [129]:
df = pd.DataFrame(columns = ['Period', 'Payment', 'Interest', 'Principal', 'Residual Balance'])
df['Period'] = dates_index

In [130]:
# monthly payment formula
# M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]

monthly_payment = loan_amnt * ((monthly_ir * (monthly_ir + 1) ** number_of_pds) / (((monthly_ir + 1) ** (number_of_pds)) - 1))
monthly_payment = round(monthly_payment, 2)
monthly_payment

2533.43

In [131]:
df['Payment'] = monthly_payment

In [132]:
# initial amounts

# first interest payment
display(loan_amnt * monthly_ir)
first_int_pmnt = loan_amnt * monthly_ir

# first payment towards principal
display(round(monthly_payment - (loan_amnt * monthly_ir), 2))
first_prin_pmnt = round(monthly_payment - (loan_amnt * monthly_ir), 2)

# residual balance after first (principal) payment
display(loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2))
first_res_bal = loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)

1875.0

658.43

499341.57

In [133]:
initial_payment = [loan_amnt * monthly_ir, round(monthly_payment - (loan_amnt * monthly_ir), 2), loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)]
initial_payment

[1875.0, 658.43, 499341.57]

In [134]:
df.loc[0, ['Interest', 'Principal', 'Residual Balance']] = initial_payment

In [135]:
int_vals = []
prin_vals = []
bal_vals = []

for i in df.index[1:, ]:
    interest = round(monthly_ir * df.iloc[i-1]['Residual Balance'], 2)
    principal = round(df.iloc[i]['Payment'] - interest, 2)
    res_bal = round(df.iloc[i-1]['Residual Balance'] - principal, 2)
    df.loc[i, ['Interest', 'Principal', 'Residual Balance']] = [interest, principal, res_bal]
    int_vals.append(interest)
    prin_vals.append(principal)
    bal_vals.append(res_bal)

In [136]:
# int_vals
# prin_vals
# bal_vals

In [141]:
# df['Residual Balance'] = df['Residual Balance'].clip(lower = 0)

In [142]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,1872.53,660.9,498680.67
2,2020-11-30,2533.43,1870.05,663.38,498017.29
3,2020-12-31,2533.43,1867.56,665.87,497351.42
4,2021-01-31,2533.43,1865.07,668.36,496683.06
...,...,...,...,...,...
355,2050-04-30,2533.43,46.96,2486.47,10036.95
356,2050-05-31,2533.43,37.64,2495.79,7541.16
357,2050-06-30,2533.43,28.28,2505.15,5036.01
358,2050-07-31,2533.43,18.89,2514.54,2521.47


In [143]:
display(df['Payment'].sum())
display(df['Interest'].sum())
display(df['Principal'].sum())

912034.8

412032.29999999993

500002.50000000006

In [155]:
# Assumptions and Inputs


loan_start_date = '09-01-2020'
loan_maturity_date = '08-31-2050'

loan_amnt = 500000
annual_ir = .045

number_of_pds = math.ceil((pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) / np.timedelta64(1, 'M'))
monthly_ir = annual_ir / 12


dates_index = pd.date_range(loan_start_date, loan_maturity_date, freq = 'M')

df = pd.DataFrame(columns = ['Period', 'Payment', 'Interest', 'Principal', 'Residual Balance'])
df['Period'] = dates_index


monthly_payment = loan_amnt * ((monthly_ir * (monthly_ir + 1) ** number_of_pds) / (((monthly_ir + 1) ** (number_of_pds)) - 1))
monthly_payment = round(monthly_payment, 2)
df['Payment'] = monthly_payment


first_int_pmnt = loan_amnt * monthly_ir
first_prin_pmnt = round(monthly_payment - (loan_amnt * monthly_ir), 2)
first_res_bal = loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)

initial_payment = [loan_amnt * monthly_ir, round(monthly_payment - (loan_amnt * monthly_ir), 2), loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)]

df.loc[0, ['Interest', 'Principal', 'Residual Balance']] = initial_payment

# df['Residual Balance'] = df['Residual Balance'].clip(lower = 0)

In [156]:
# creating function
int_vals = []
prin_vals = []
bal_vals = []

for i in df.index[1:, ]:
    interest = round(monthly_ir * df.iloc[i-1]['Residual Balance'], 2)
    principal = round(df.iloc[i]['Payment'] - interest, 2)
    res_bal = round(df.iloc[i-1]['Residual Balance'] - principal, 2)
    df.loc[i, ['Interest', 'Principal', 'Residual Balance']] = [interest, principal, res_bal]
    int_vals.append(interest)
    prin_vals.append(principal)
    bal_vals.append(res_bal)

In [157]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,1872.53,660.9,498680.67
2,2020-11-30,2533.43,1870.05,663.38,498017.29
3,2020-12-31,2533.43,1867.56,665.87,497351.42
4,2021-01-31,2533.43,1865.07,668.36,496683.06
...,...,...,...,...,...
355,2050-04-30,2533.43,46.96,2486.47,10036.95
356,2050-05-31,2533.43,37.64,2495.79,7541.16
357,2050-06-30,2533.43,28.28,2505.15,5036.01
358,2050-07-31,2533.43,18.89,2514.54,2521.47


In [158]:
int_vals = []
prin_vals = []
bal_vals = []

def create_pmnt_df(df):
    for i in df.index[1:, ]:
        interest = round(monthly_ir * df.iloc[i-1]['Residual Balance'], 2)
        principal = round(df.iloc[i]['Payment'] - interest, 2)
        res_bal = round(df.iloc[i-1]['Residual Balance'] - principal, 2)
        df.loc[i, ['Interest', 'Principal', 'Residual Balance']] = [interest, principal, res_bal]
        int_vals.append(interest)
        prin_vals.append(principal)
        bal_vals.append(res_bal)
    return df

In [159]:
create_pmnt_df(df)

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,1872.53,660.9,498680.67
2,2020-11-30,2533.43,1870.05,663.38,498017.29
3,2020-12-31,2533.43,1867.56,665.87,497351.42
4,2021-01-31,2533.43,1865.07,668.36,496683.06
...,...,...,...,...,...
355,2050-04-30,2533.43,46.96,2486.47,10036.95
356,2050-05-31,2533.43,37.64,2495.79,7541.16
357,2050-06-30,2533.43,28.28,2505.15,5036.01
358,2050-07-31,2533.43,18.89,2514.54,2521.47


In [ ]:
# creating full function

### Sim 2 - Payments on Two Loans

In [160]:
# int_vals = []
# prin_vals = []
# bal_vals = []

def create_pmnt_df(df):
    for i in df.index[1:, ]:
        interest = round(monthly_ir * df.iloc[i-1]['Residual Balance'], 2)
        principal = round(df.iloc[i]['Payment'] - interest, 2)
        res_bal = round(df.iloc[i-1]['Residual Balance'] - principal, 2)
        df.loc[i, ['Interest', 'Principal', 'Residual Balance']] = [interest, principal, res_bal]
#         int_vals.append(interest)
#         prin_vals.append(principal)
#         bal_vals.append(res_bal)
    return df

In [182]:
# Assumptions and Inputs
# lets use specified start and end dates for the payments on each loan


#loan 1
loan_start_date = '09-01-2020'
loan_maturity_date = '08-31-2025'

loan_amnt = 500000
annual_ir = .045

number_of_pds = math.ceil((pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) / np.timedelta64(1, 'M'))
monthly_ir = annual_ir / 12


dates_index = pd.date_range(loan_start_date, loan_maturity_date, freq = 'M')

df = pd.DataFrame(columns = ['Period', 'Payment', 'Interest', 'Principal', 'Residual Balance'])
df['Period'] = dates_index
df['Tranche'] = 'A'


monthly_payment = loan_amnt * ((monthly_ir * (monthly_ir + 1) ** number_of_pds) / (((monthly_ir + 1) ** (number_of_pds)) - 1))
monthly_payment = round(monthly_payment, 2)
df['Payment'] = monthly_payment


first_int_pmnt = round(loan_amnt * monthly_ir, 2)
first_prin_pmnt = round(monthly_payment - (loan_amnt * monthly_ir), 2)
first_res_bal = loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)

initial_payment = [first_int_pmnt, first_prin_pmnt, first_res_bal]

df.loc[0, ['Interest', 'Principal', 'Residual Balance']] = initial_payment

# df['Residual Balance'] = df['Residual Balance'].clip(lower = 0)

loan_1 = create_pmnt_df(df)
loan_1

,Period,Payment,Interest,Principal,Residual Balance,Tranche
0,2020-09-30,9321.51,1875.0,7446.51,492553.49,A
1,2020-10-31,9321.51,1847.08,7474.43,485079.06,A
2,2020-11-30,9321.51,1819.05,7502.46,477576.6,A
3,2020-12-31,9321.51,1790.91,7530.6,470046.0,A
4,2021-01-31,9321.51,1762.67,7558.84,462487.16,A
5,2021-02-28,9321.51,1734.33,7587.18,454899.98,A
6,2021-03-31,9321.51,1705.87,7615.64,447284.34,A
7,2021-04-30,9321.51,1677.32,7644.19,439640.15,A
8,2021-05-31,9321.51,1648.65,7672.86,431967.29,A
9,2021-06-30,9321.51,1619.88,7701.63,424265.66,A


In [181]:
# Assumptions and Inputs
# lets use specified start and end dates for the payments on each loan


#loan 2
loan_start_date = '09-01-2025'
loan_maturity_date = '08-31-2030'

loan_amnt = 250000
annual_ir = .065

number_of_pds = math.ceil((pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) / np.timedelta64(1, 'M'))
monthly_ir = annual_ir / 12


dates_index = pd.date_range(loan_start_date, loan_maturity_date, freq = 'M')

df = pd.DataFrame(columns = ['Period', 'Payment', 'Interest', 'Principal', 'Residual Balance'])
df['Period'] = dates_index
df['Tranche'] = 'B'

monthly_payment = loan_amnt * ((monthly_ir * (monthly_ir + 1) ** number_of_pds) / (((monthly_ir + 1) ** (number_of_pds)) - 1))
monthly_payment = round(monthly_payment, 2)
df['Payment'] = monthly_payment


first_int_pmnt = round(loan_amnt * monthly_ir, 2)
first_prin_pmnt = round(monthly_payment - (loan_amnt * monthly_ir), 2)
first_res_bal = loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)

initial_payment = [first_int_pmnt, first_prin_pmnt, first_res_bal]

df.loc[0, ['Interest', 'Principal', 'Residual Balance']] = initial_payment

# df['Residual Balance'] = df['Residual Balance'].clip(lower = 0)

loan_2 = create_pmnt_df(df)
loan_2

,Period,Payment,Interest,Principal,Residual Balance,Tranche
0,2025-09-30,4891.54,1354.17,3537.37,246462.63,B
1,2025-10-31,4891.54,1335.01,3556.53,242906.1,B
2,2025-11-30,4891.54,1315.74,3575.8,239330.3,B
3,2025-12-31,4891.54,1296.37,3595.17,235735.13,B
4,2026-01-31,4891.54,1276.9,3614.64,232120.49,B
5,2026-02-28,4891.54,1257.32,3634.22,228486.27,B
6,2026-03-31,4891.54,1237.63,3653.91,224832.36,B
7,2026-04-30,4891.54,1217.84,3673.7,221158.66,B
8,2026-05-31,4891.54,1197.94,3693.6,217465.06,B
9,2026-06-30,4891.54,1177.94,3713.6,213751.46,B


In [187]:
pd.options.display.max_rows = 200
result_1 = loan_1.append(loan_2)

In [188]:
loan_1.head()

,Period,Payment,Interest,Principal,Residual Balance,Tranche
0,2020-09-30,9321.51,1875.0,7446.51,492553.49,A
1,2020-10-31,9321.51,1847.08,7474.43,485079.06,A
2,2020-11-30,9321.51,1819.05,7502.46,477576.6,A
3,2020-12-31,9321.51,1790.91,7530.6,470046.0,A
4,2021-01-31,9321.51,1762.67,7558.84,462487.16,A


In [189]:
loan_2.head()

,Period,Payment,Interest,Principal,Residual Balance,Tranche
0,2025-09-30,4891.54,1354.17,3537.37,246462.63,B
1,2025-10-31,4891.54,1335.01,3556.53,242906.1,B
2,2025-11-30,4891.54,1315.74,3575.8,239330.3,B
3,2025-12-31,4891.54,1296.37,3595.17,235735.13,B
4,2026-01-31,4891.54,1276.9,3614.64,232120.49,B


In [ ]:
### Sim 3 - Payments on Threee Loans

In [ ]:
### Sim 4 - Payments on Three Loans + Equity Piece

In [ ]:
### Sim 5 - Payments on Three Loans + Equity Piece, with Prepayments